# Converting p-values to probabilities
- in my last blog post, I gave a short introduction to Bayesian analysis
- in this blog post, i will 

In [106]:
# generate data for treatment and control
import numpy as np
import scipy.stats as stats

control_mean = .6
effect_size = .15
size_t = 40
size_c = 30

# generate some data and do a test of equality.  repeat until i get a p-value that i like
dummy = True
while dummy == True:
    control = np.random.binomial(1,control_mean,size_c)
    treatment = np.random.binomial(1,control_mean+effect_size, size_t)

    # estimate the p-value for a t-test
    result = stats.ttest_ind_from_stats(treatment.mean(),treatment.std(), size_t, control.mean(),control.std(), size_c) 
    if result[1] > .15 and result[1] < .2:
        dummy = False

0.01465222841
0.445675402483
0.216817627258
0.334331594389
0.00699069661662
0.0285852681968
0.599110691371
0.00260287577617
0.0682918043251
0.179706168345


In [136]:
# combine data into a single vector and add an array for 
treat_status = np.repeat(np.array([0,1]), [size_c, size_t])
outcome = np.concatenate((treatment,control),0)

In [145]:
# estimate the data using a model
# import the pystan package.  this package allows python to call the Stan language
import pystan
# The next few lines define the model in Stan.  The entire Stan model is saved as the python variable
# stan_code which I use later to fit the model. 
stan_code = '''
data {
    int<lower=0> N;  // number of observations
    int<lower=0,upper=1> y[N];  // outcome data
    vector[N] treat;  // variable indicating treatment arm for each observation
}
parameters {
    real alpha;
    real beta;
}
model {
    alpha ~ normal(0,2);  // this means that our prior is that the control mean is .5  
    beta ~ normal(0,1); // 
    y ~ bernoulli_logit(alpha + beta*treat);
}
'''
stan_data = {'N': len(outcome),
        'y': outcome,
        'treat': treat_status }

In [146]:
fit = pystan.stan(model_code=stan_code, data=stan_data, iter=1000, chains=4)

/Users/douglasjohnson/anaconda/lib/python3.5/multiprocessing/reduction.py:50: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  cls(buf, protocol).dump(obj)
/Users/douglasjohnson/anaconda/lib/python3.5/multiprocessing/reduction.py:50: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  cls(buf, protocol).dump(obj)
/Users/douglasjohnson/anaconda/lib/python3.5/multiprocessing/reduction.py:50: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  cls(buf, protocol).dump(obj)
/Users/douglasjohnson/anaconda/lib/python3.5/multiprocessing/reduction.py:50: UserWarning: Pic

In [148]:
print(fit)

Inference for Stan model: anon_model_b71f87ef6e649cc3506bc4de66c524ab.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha   0.68  8.0e-3   0.36   0.02   0.43   0.68   0.92    1.4   2000   1.01
beta    0.17  9.6e-3   0.43  -0.63  -0.13   0.18   0.47    1.0   2000   1.01
lp__  -44.56    0.02   0.95 -47.17 -44.94 -44.28  -43.9 -43.63   2000    1.0

Samples were drawn using NUTS(diag_e) at Mon Jun 27 18:10:09 2016.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).
